DEEP LEARNING MODEL -  LSTM, 1D CNNs

In [2]:
pip install tensorflow

   ---------------------------------------- 0.0/390.3 MB ? eta -:--:--
   ---------------------------------------- 1.0/390.3 MB 6.3 MB/s eta 0:01:02
   ---------------------------------------- 2.1/390.3 MB 5.3 MB/s eta 0:01:13
   ---------------------------------------- 2.9/390.3 MB 5.1 MB/s eta 0:01:17
   ---------------------------------------- 3.9/390.3 MB 4.8 MB/s eta 0:01:21
   ---------------------------------------- 4.7/390.3 MB 4.8 MB/s eta 0:01:20
    --------------------------------------- 5.8/390.3 MB 4.7 MB/s eta 0:01:22
    --------------------------------------- 6.8/390.3 MB 4.7 MB/s eta 0:01:22
    --------------------------------------- 7.9/390.3 MB 4.7 MB/s eta 0:01:21
    --------------------------------------- 8.7/390.3 MB 4.5 MB/s eta 0:01:24
    --------------------------------------- 9.2/390.3 MB 4.5 MB/s eta 0:01:26
   - -------------------------------------- 10.0/390.3 MB 4.3 MB/s eta 0:01:28
   - -------------------------------------- 10.5/390.3 MB 4.3 MB/s eta

In [1]:

import os  
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import Precision, Recall, F1Score
from tensorflow.keras.callbacks import EarlyStopping

In [2]:

def download_and_extract_data(url, extract_to):
    import urllib.request
    import zipfile
    if not os.path.exists(extract_to):
        os.makedirs(extract_to)
    zip_path = os.path.join(extract_to, "data.zip")
    urllib.request.urlretrieve(url, zip_path)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    os.remove(zip_path)


def load_raw_data(data_dir):
  
    sensor_files = [
        'total_acc_x_train.txt', 'total_acc_y_train.txt', 'total_acc_z_train.txt',
        'body_gyro_x_train.txt', 'body_gyro_y_train.txt', 'body_gyro_z_train.txt'
    ]
    sensor_data = []
    for file in sensor_files:
        file_path = os.path.join(data_dir, "train", "Inertial Signals", file)
        data = pd.read_csv(file_path, delim_whitespace=True, header=None)
        sensor_data.append(data)
   
    sensor_data = np.concatenate(sensor_data, axis=1)

    label_path = os.path.join(data_dir, "train", "y_train.txt")
    labels = pd.read_csv(label_path, delim_whitespace=True, header=None)
    return sensor_data, labels


def preprocess_data(data, labels, window_size=128, step_size=64):

    data = pd.DataFrame(data).fillna(method='ffill').values 
   
    scaler = StandardScaler()
    data = scaler.fit_transform(data)


    segments = []
    labels_segmented = []
    for i in range(0, len(data) - window_size, step_size):
        segment = data[i:i + window_size]
        label = labels.values[i + window_size]
        segments.append(segment)
        labels_segmented.append(label)
    return np.array(segments), np.array(labels_segmented)


def create_lstm_model(input_shape, num_classes):
    model = Sequential()
    model.add(LSTM(64, input_shape=input_shape, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(32, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', Precision(), Recall(), F1Score()])
    return model


def create_cnn_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', Precision(), Recall(), F1Score()])
    return model


def train_and_evaluate_model(model, X_train, y_train, X_test, y_test, epochs=50, batch_size=64):
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                        validation_split=0.2, callbacks=[early_stopping])

    results = model.evaluate(X_test, y_test, batch_size=batch_size)
    print(f"Test Accuracy: {results[1]}, Precision: {results[2]}, Recall: {results[3]}, F1-Score: {results[4]}")
    return history

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import zipfile
import urllib.request
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, MaxPooling1D, Flatten


def download_and_extract_dataset(url, extract_path):

    if not os.path.exists(extract_path):
        os.makedirs(extract_path)
    

    zip_path = os.path.join(extract_path, "usr.zip")
    if not os.path.exists(zip_path):
        print("Downloading dataset...")
        urllib.request.urlretrieve(url, zip_path)
    
 
    if not os.path.exists(os.path.join(extract_path, "UCI HAR Dataset")):
        print("Extracting dataset...")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)
    
    print("Dataset ready at:", os.path.join(extract_path, "UCI HAR Dataset"))
    return os.path.join(extract_path, "UCI HAR Dataset")

def load_raw_inertial_data(dataset_path):
    """
    Load only the raw inertial sensor data (accelerometer & gyroscope readings)
    """

    train_acc_path = os.path.join(dataset_path, "train", "Inertial Signals", "total_acc_x_train.txt")
    train_gyro_path = os.path.join(dataset_path, "train", "Inertial Signals", "body_gyro_x_train.txt")
    test_acc_path = os.path.join(dataset_path, "test", "Inertial Signals", "total_acc_x_test.txt")
    test_gyro_path = os.path.join(dataset_path, "test", "Inertial Signals", "body_gyro_x_test.txt")
    
  
    for file_path in [train_acc_path, train_gyro_path, test_acc_path, test_gyro_path]:
        if not os.path.exists(file_path):
            print(f"Warning: File {file_path} does not exist!")
    

    axes = ['x', 'y', 'z']
    
  
    train_acc_data = {}
    train_gyro_data = {}
    test_acc_data = {}
    test_gyro_data = {}
    

    for axis in axes:

        file_path = os.path.join(dataset_path, "train", "Inertial Signals", f"total_acc_{axis}_train.txt")
        train_acc_data[axis] = np.loadtxt(file_path)
        
       
        file_path = os.path.join(dataset_path, "train", "Inertial Signals", f"body_gyro_{axis}_train.txt")
        train_gyro_data[axis] = np.loadtxt(file_path)

    for axis in axes:

        file_path = os.path.join(dataset_path, "test", "Inertial Signals", f"total_acc_{axis}_test.txt")
        test_acc_data[axis] = np.loadtxt(file_path)
        
    
        file_path = os.path.join(dataset_path, "test", "Inertial Signals", f"body_gyro_{axis}_test.txt")
        test_gyro_data[axis] = np.loadtxt(file_path)
    

    train_labels = np.loadtxt(os.path.join(dataset_path, "train", "y_train.txt")).astype(int)
    test_labels = np.loadtxt(os.path.join(dataset_path, "test", "y_test.txt")).astype(int)
    
    
    n_train_samples = train_acc_data['x'].shape[0]
    n_timesteps = train_acc_data['x'].shape[1]
    
    train_acc_combined = np.zeros((n_train_samples, n_timesteps, 3))
    train_gyro_combined = np.zeros((n_train_samples, n_timesteps, 3))
    
    for i, axis in enumerate(axes):
        train_acc_combined[:, :, i] = train_acc_data[axis]
        train_gyro_combined[:, :, i] = train_gyro_data[axis]
    

    n_test_samples = test_acc_data['x'].shape[0]
    
    test_acc_combined = np.zeros((n_test_samples, n_timesteps, 3))
    test_gyro_combined = np.zeros((n_test_samples, n_timesteps, 3))
    
    for i, axis in enumerate(axes):
        test_acc_combined[:, :, i] = test_acc_data[axis]
        test_gyro_combined[:, :, i] = test_gyro_data[axis]
    
    print(f"Loaded {n_train_samples} training samples and {n_test_samples} test samples")
    print(f"Each sample has {n_timesteps} time steps")
    
    return {
        'train_acc': train_acc_combined,
        'train_gyro': train_gyro_combined,
        'test_acc': test_acc_combined,
        'test_gyro': test_gyro_combined,
        'train_labels': train_labels,
        'test_labels': test_labels
    }

def preprocess_data(data, normalize=True):
  
    train_acc = data['train_acc']
    train_gyro = data['train_gyro']
    test_acc = data['test_acc']
    test_gyro = data['test_gyro']
    

    print("Checking for missing values...")
    print(f"NaN in train_acc: {np.isnan(train_acc).sum()}")
    print(f"NaN in train_gyro: {np.isnan(train_gyro).sum()}")
    print(f"NaN in test_acc: {np.isnan(test_acc).sum()}")
    print(f"NaN in test_gyro: {np.isnan(test_gyro).sum()}")
    

    if np.isnan(train_acc).sum() > 0:
        for i in range(train_acc.shape[2]):
            channel_mean = np.nanmean(train_acc[:, :, i])
            train_acc[:, :, i] = np.nan_to_num(train_acc[:, :, i], nan=channel_mean)
    
    if np.isnan(train_gyro).sum() > 0:
        for i in range(train_gyro.shape[2]):
            channel_mean = np.nanmean(train_gyro[:, :, i])
            train_gyro[:, :, i] = np.nan_to_num(train_gyro[:, :, i], nan=channel_mean)
    
    if np.isnan(test_acc).sum() > 0:
        for i in range(test_acc.shape[2]):
            channel_mean = np.nanmean(test_acc[:, :, i])
            test_acc[:, :, i] = np.nan_to_num(test_acc[:, :, i], nan=channel_mean)
    
    if np.isnan(test_gyro).sum() > 0:
        for i in range(test_gyro.shape[2]):
            channel_mean = np.nanmean(test_gyro[:, :, i])
            test_gyro[:, :, i] = np.nan_to_num(test_gyro[:, :, i], nan=channel_mean)
    

    if normalize:
        print("Normalizing data...")
     
        train_acc_shape = train_acc.shape
        train_gyro_shape = train_gyro.shape
        test_acc_shape = test_acc.shape
        test_gyro_shape = test_gyro.shape
        
        train_acc_reshaped = train_acc.reshape(-1, train_acc.shape[2])
        train_gyro_reshaped = train_gyro.reshape(-1, train_gyro.shape[2])
        test_acc_reshaped = test_acc.reshape(-1, test_acc.shape[2])
        test_gyro_reshaped = test_gyro.reshape(-1, test_gyro.shape[2])
        
   
        scaler_acc = StandardScaler()
        scaler_gyro = StandardScaler()
        
        train_acc_normalized = scaler_acc.fit_transform(train_acc_reshaped)
        test_acc_normalized = scaler_acc.transform(test_acc_reshaped)
        
        train_gyro_normalized = scaler_gyro.fit_transform(train_gyro_reshaped)
        test_gyro_normalized = scaler_gyro.transform(test_gyro_reshaped)
        

        train_acc = train_acc_normalized.reshape(train_acc_shape)
        train_gyro = train_gyro_normalized.reshape(train_gyro_shape)
        test_acc = test_acc_normalized.reshape(test_acc_shape)
        test_gyro = test_gyro_normalized.reshape(test_gyro_shape)
    

    train_data = np.concatenate([train_acc, train_gyro], axis=2)
    test_data = np.concatenate([test_acc, test_gyro], axis=2)
    
    print(f"Final training data shape: {train_data.shape}")
    print(f"Final test data shape: {test_data.shape}")
    

    train_labels = tf.keras.utils.to_categorical(data['train_labels'] - 1)  # Subtract 1 to make labels 0-indexed
    test_labels = tf.keras.utils.to_categorical(data['test_labels'] - 1)
    
    return train_data, test_data, train_labels, test_labels

def visualize_data(data, labels, activities, num_samples=3):
 
    activity_names = ['Walking', 'Walking Upstairs', 'Walking Downstairs', 'Sitting', 'Standing', 'Laying']
    

    plt.figure(figsize=(15, 20))
    
    for i, activity in enumerate(activities):
       
        activity_indices = np.where(labels == activity)[0]
        
      
        selected_indices = np.random.choice(activity_indices, min(num_samples, len(activity_indices)), replace=False)
        
        for j, idx in enumerate(selected_indices):
            plt.subplot(len(activities), num_samples, i * num_samples + j + 1)
            

            plt.plot(data[idx, :, 0], 'r-', label='Acc X')
            plt.plot(data[idx, :, 1], 'g-', label='Acc Y')
            plt.plot(data[idx, :, 2], 'b-', label='Acc Z')
            

            plt.plot(data[idx, :, 3], 'r--', label='Gyro X')
            plt.plot(data[idx, :, 4], 'g--', label='Gyro Y')
            plt.plot(data[idx, :, 5], 'b--', label='Gyro Z')
            
            if j == 0:
                plt.ylabel(activity_names[activity-1])
            
            if i == 0:
                plt.title(f"Sample {j+1}")
            
            if i == len(activities) - 1 and j == 0:
                plt.legend(loc='upper center', bbox_to_anchor=(1.5, -0.05), ncol=6)
    
    plt.tight_layout()
    plt.savefig('har_data_visualization.png')
    plt.close()
    print("Visualization saved as 'har_data_visualization.png'")

# 2. Deep Learning Models
def build_lstm_model(input_shape, num_classes):
    
    model = Sequential([
        LSTM(64, input_shape=input_shape, return_sequences=True),
        Dropout(0.2),
        LSTM(32),
        Dropout(0.2),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(
        loss='categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )
    
    print("LSTM Model Summary:")
    model.summary()
    
    return model

def build_cnn_lstm_model(input_shape, num_classes):
   
    model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        Dropout(0.2),
        Conv1D(filters=32, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Dropout(0.2),
        LSTM(32, return_sequences=False),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(
        loss='categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )
    
    print("CNN-LSTM Model Summary:")
    model.summary()
    
    return model

def train_and_evaluate_model(model, train_data, train_labels, test_data, test_labels, epochs=50, batch_size=32):
   
    X_train, X_val, y_train, y_val = train_test_split(
        train_data, train_labels, test_size=0.2, random_state=42
    )
    

    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=10, restore_best_weights=True
    )
    
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001
    )
    

    history = model.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping, reduce_lr],
        verbose=1
    )
    
 
    test_loss, test_accuracy = model.evaluate(test_data, test_labels, verbose=0)
    print(f"Test Accuracy: {test_accuracy:.4f}")
    
    
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='lower right')
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper right')
    
    plt.tight_layout()
    plt.savefig('training_history.png')
    plt.close()
    
 
    y_pred = np.argmax(model.predict(test_data), axis=1)
    y_true = np.argmax(test_labels, axis=1)
    

    print("\nClassification Report:")
    target_names = ['Walking', 'Walking Upstairs', 'Walking Downstairs', 'Sitting', 'Standing', 'Laying']
    print(classification_report(y_true, y_pred, target_names=target_names))
    
  
    conf_mat = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues',
                xticklabels=target_names, yticklabels=target_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.savefig('confusion_matrix.png')
    plt.close()
    
    return history, y_pred, y_true

def main():

    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip"
    extract_path = "./data"
    
   
    dataset_path = download_and_extract_dataset(url, extract_path)
    

    data = load_raw_inertial_data(dataset_path)
    

    train_data, test_data, train_labels, test_labels = preprocess_data(data)
    

    visualize_data(train_data, data['train_labels'], [1, 2, 3, 4, 5, 6], num_samples=3)
    
    
    input_shape = (train_data.shape[1], train_data.shape[2])  # Corrected indentation
    num_classes = train_labels.shape[1]
    
    print("\n=== Training LSTM Model ===")
    lstm_model = build_lstm_model(input_shape, num_classes)
    lstm_history, lstm_y_pred, lstm_y_true = train_and_evaluate_model(
        lstm_model, train_data, train_labels, test_data, test_labels
    )
    
   
    print("\n=== Training CNN-LSTM Model ===")
    cnn_lstm_model = build_cnn_lstm_model(input_shape, num_classes)
    cnn_lstm_history, cnn_lstm_y_pred, cnn_lstm_y_true = train_and_evaluate_model(
        cnn_lstm_model, train_data, train_labels, test_data, test_labels
    )
    
   
    print("\n=== Model Comparison ===")
    
    
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    

    lstm_accuracy = accuracy_score(lstm_y_true, lstm_y_pred)
    lstm_precision = precision_score(lstm_y_true, lstm_y_pred, average='weighted')
    lstm_recall = recall_score(lstm_y_true, lstm_y_pred, average='weighted')
    lstm_f1 = f1_score(lstm_y_true, lstm_y_pred, average='weighted')
    
  
    cnn_lstm_accuracy = accuracy_score(cnn_lstm_y_true, cnn_lstm_y_pred)
    cnn_lstm_precision = precision_score(cnn_lstm_y_true, cnn_lstm_y_pred, average='weighted')
    cnn_lstm_recall = recall_score(cnn_lstm_y_true, cnn_lstm_y_pred, average='weighted')
    cnn_lstm_f1 = f1_score(cnn_lstm_y_true, cnn_lstm_y_pred, average='weighted')
    
   
    print(f"LSTM - Accuracy: {lstm_accuracy:.4f}, Precision: {lstm_precision:.4f}, Recall: {lstm_recall:.4f}, F1: {lstm_f1:.4f}")
    print(f"CNN-LSTM - Accuracy: {cnn_lstm_accuracy:.4f}, Precision: {cnn_lstm_precision:.4f}, Recall: {cnn_lstm_recall:.4f}, F1: {cnn_lstm_f1:.4f}")
    
  
    lstm_model.save('lstm_model.h5')
    cnn_lstm_model.save('cnn_lstm_model.h5')
    print("Models saved as 'lstm_model.h5' and 'cnn_lstm_model.h5'")
if __name__ == "__main__":
    main()

Dataset ready at: ./data\UCI HAR Dataset
Loaded 7352 training samples and 2947 test samples
Each sample has 128 time steps
Checking for missing values...
NaN in train_acc: 0
NaN in train_gyro: 0
NaN in test_acc: 0
NaN in test_gyro: 0
Normalizing data...
Final training data shape: (7352, 128, 6)
Final test data shape: (2947, 128, 6)
Visualization saved as 'har_data_visualization.png'

=== Training LSTM Model ===
LSTM Model Summary:


C:\Users\V.SHRI KAMALESH\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 128, 64)        │        18,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │           198 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 30,790 (120.27 KB)

...

 Trainable params: 15,910 (62.15 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
184/184 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.5535 - loss: 1.1148 - val_accuracy: 0.8620 - val_loss: 0.3719 - learning_rate: 0.0010
Epoch 2/50
184/184 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.8749 - loss: 0.3533 - val_accuracy: 0.8892 - val_loss: 0.2782 - learning_rate: 0.0010
Epoch 3/50
184/184 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9271 - loss: 0.2216 - val_accuracy: 0.9395 - val_loss: 0.1688 - learning_rate: 0.0010
Epoch 4/50
184/184 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.9373 - loss: 0.1779 - val_accuracy: 0.9524 - val_loss: 0.1354 - learning_rate: 0.0010
Epoch 5/50
184/184 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.9450 - loss: 0.1511 - val_accuracy: 0.9531 - val_loss: 0.1290 - learning_rate: 0.0010
Epoch 6/50
184/184 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.9566 - loss: 0.1263 - val_accuracy: 0.9354 - val_loss: 0.1634 - learning_rate: 0.0010
Epoch 7/50
184/184 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.9465 - loss: 0.

LSTM - Accuracy: 0.9002, Precision: 0.9031, Recall: 0.9002, F1: 0.9001
CNN-LSTM - Accuracy: 0.9108, Precision: 0.9104, Recall: 0.9108, F1: 0.9101


Models saved as 'lstm_model.h5' and 'cnn_lstm_model.h5'
